In [108]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score 

In [2]:
data = pd.read_excel("data.xlsx")
distances = pd.read_excel("distances.xlsx")

In [88]:
distances = distances.set_index('Unnamed: 0')
distances.head()

,89636817964,89636817923,89636817955,совкомбанк горячая линия,совкомбанк вологда телефон,совкомбанк вологда адреса и телефоны,вк,оплата теле2 с банковской карты вологда,погода кубенское,оплата теле2 с банковской карты Вологда,...,инсталляция с унитазом в комплекте купить,светильник подвесной цилиндр,настенный ламинат,декор панно,тв шоу,абстракция для интерьера,"Стул барный DOBRIN MIRA, LM-5001, чёрный",lost ark зачем золото,not bad now you gif e,Брюсов биография
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
89636817964,-2.220446e-16,0.015663,0.018285,0.516425,0.568367,0.572036,0.341589,0.508128,0.566768,0.508128,...,0.503163,0.538192,0.629471,0.521013,0.529668,0.600208,0.463813,0.619172,0.544156,0.556837
89636817923,1.566285e-02,0.000000,0.011698,0.500336,0.527769,0.532899,0.314822,0.478625,0.556110,0.478625,...,0.519810,0.531629,0.614651,0.526390,0.543031,0.601169,0.436581,0.621279,0.539377,0.564608
89636817955,1.828505e-02,0.011698,0.000000,0.488724,0.503735,0.505728,0.321093,0.434377,0.530669,0.434377,...,0.522956,0.530611,0.620084,0.513078,0.571157,0.595296,0.436119,0.645579,0.549280,0.565716
совкомбанк горячая линия,5.164252e-01,0.500336,0.488724,0.000000,0.146539,0.152986,0.486947,0.406711,0.613810,0.406711,...,0.630024,0.749376,0.738606,0.800993,0.645955,0.817137,0.618561,0.736538,0.715732,0.564268
совкомбанк вологда телефон,5.683672e-01,0.527769,0.503735,0.146539,0.000000,0.014545,0.588330,0.285385,0.515219,0.285385,...,0.765507,0.743573,0.769575,0.874331,0.763829,0.891492,0.628036,0.743013,0.831801,0.575721


In [12]:
data['last_click_ts'] = data['last_click_ts'].fillna(0)

In [58]:
class Request(object):
    def __init__(self, request):
        self.user_id = request["user_id"]
        self.id = request["request_id"]
        self.query = request["query"]
        self.ts = request["ts"]
        self.last_click_ts = max(request["ts"], request["last_click_ts"])
        self.is_reload = request["is_reload"]
        self.is_not_zero_page = request["is_not_zero_page"]
        self.has_click = request["is_not_zero_page"]
        self.exception_flag = request["exception_flag"]
        self.session_id = None

In [132]:
MAX_MINUTES_OF_INACTIVITY = 30
MAX_MINUTES_OF_INACTIVITY_FOR_DISTANT_QUERIES = 30
DISTANT_QUERIES_THRESHOLD = 0.4

class Session(object):
    def __init__(self, request):
        self.user_id = request.user_id
        self.first_request_id = request.id
        self.ts_end = None
        self.last_query = None

        self.append(request)

    def append(self, request):
        self.ts_end = request.last_click_ts
        self.last_query = request.query

    def is_in_session(self, request):
        if request.is_not_zero_page:  # не первая страница - всегда в той же сессии
            return True

        if request.ts > self.ts_end + 60 * MAX_MINUTES_OF_INACTIVITY:
            return False
        if request.ts > self.ts_end + 60 * MAX_MINUTES_OF_INACTIVITY_FOR_DISTANT_QUERIES:
            if get_query_dist(self.last_query, request.query) > DISTANT_QUERIES_THRESHOLD:
                return False
        return True

    def get_name(self):
        return str(self.user_id) + "-" + str(self.first_request_id)


def is_technical_request(request):
    if type(request.query) != str:
        return True
    if not request.has_click:
        if request.exception_flag or request.is_reload:
            return True
    return False



def add_session_data(requests):
    sessions = list()
    cur_session_id = ""
    for request in requests:
        # Skip technical requests
        if is_technical_request(request) and False:
            request.session_id = cur_session_id
            continue
        if len(sessions) == 0 or not sessions[-1].is_in_session(request):
            sessions.append(Session(request))
            cur_session_id = sessions[-1].get_name()
        else:
            sessions[-1].append(request)
        request.session_id = cur_session_id

    if len(sessions) > 0:
        for request in requests:
            if request.session_id == "":
                request.session_id = sessions[0].get_name()
            else:
                break

def get_query_dist(q1, q2):
    # print(q1, q2)
    return distances.loc[q1, q2]

In [133]:
sessions = []
for uid in set(data['user_id']):
    requests = []
    for index, row in data[data['user_id'] == uid].sort_values('ts').iterrows():
        request = Request(row)
        requests.append(request)
    add_session_data(requests)
    sessions = sessions + [{'request_id': vars(r)['id'], 'session_id': vars(r)['session_id']} for r in requests]

In [119]:
results = pd.DataFrame(sessions).merge(data)

In [122]:
results = pd.DataFrame(sessions).merge(results, suffixes=('_without_dist', ''), left_on=['request_id'], right_on=['request_id'])

In [134]:
results = pd.DataFrame(sessions).merge(results, suffixes=('_without_dist_and_tech', ''), left_on=['request_id'], right_on=['request_id'])

In [135]:
results.head()

,request_id,session_id_without_dist_and_tech,session_id_without_dist,session_id,Unnamed: 0,user_id,query,ts,last_click_ts,is_reload,is_not_zero_page,has_click,exception_flag,verdict,sample,y_true,y_pred,y_pred_without_dist
0,251,0-251,0-251,0-251,899,0,ипотека кредит проценты узбекистан,1701420565,1.701421e+09,False,False,True,False,начало,kpi,True,True,True
1,2967,0-2967,0-2967,0-2967,900,0,Perfect breasts Thai girl fucked by Japan man,1701524468,1.701524e+09,False,False,True,False,начало,kpi,True,True,True
2,2538,0-2538,0-2538,0-2538,901,0,порно туретиский анал,1701732310,1.701732e+09,False,False,True,False,начало,kpi,True,True,True
3,850,0-850,0-850,0-850,902,0,душевая кабина смеситель от пола размер,1701756963,1.701757e+09,False,False,True,False,начало,kpi,True,True,True
4,2559,1-2559,1-2559,1-2559,1844,1,дома с женой секс,1701244303,1.701244e+09,False,False,True,False,начало,validate,True,True,True


In [136]:
results['y_true'] = results['verdict'] == 'начало'
results['y_pred'] = results.apply(lambda x: x['session_id'].split('-')[-1] == str(x['request_id']), axis=1)
results['y_pred_without_dist'] = results.apply(lambda x: x['session_id_without_dist'].split('-')[-1] == str(x['request_id']), axis=1)
results['y_pred_without_dist_and_tech'] = results.apply(lambda x: x['session_id_without_dist_and_tech'].split('-')[-1] == str(x['request_id']), axis=1)

In [137]:
f1_score(results[results['sample'] == 'validate']['y_true'], results[results['sample'] == 'validate']['y_pred'])

0.8845789971617786

In [141]:
f1_score(results[results['sample'] == 'validate']['y_true'], results[results['sample'] == 'validate']['y_pred_without_dist'])

0.8282520325203252

In [142]:
f1_score(results[results['sample'] == 'validate']['y_true'], results[results['sample'] == 'validate']['y_pred_without_dist_and_tech'])

0.798219584569733

In [138]:
f1_score(results[results['sample'] == 'kpi']['y_true'], results[results['sample'] == 'kpi']['y_pred'])

0.925531914893617

In [139]:
f1_score(results[results['sample'] == 'kpi']['y_true'], results[results['sample'] == 'kpi']['y_pred_without_dist'])

0.8815028901734104

In [140]:
f1_score(results[results['sample'] == 'kpi']['y_true'], results[results['sample'] == 'kpi']['y_pred_without_dist_and_tech'])

0.8559201141226819

In [127]:
precision_score(results[results['sample'] == 'kpi']['y_true'], results[results['sample'] == 'kpi']['y_pred'])

0.9329758713136729

In [128]:
recall_score(results[results['sample'] == 'kpi']['y_true'], results[results['sample'] == 'kpi']['y_pred'])

0.9182058047493403

In [130]:
precision_score(results[results['sample'] == 'kpi']['y_true'], results[results['sample'] == 'kpi']['y_pred_without_dist'])

0.9744408945686901

In [131]:
recall_score(results[results['sample'] == 'kpi']['y_true'], results[results['sample'] == 'kpi']['y_pred_without_dist'])

0.8047493403693932

In [143]:
precision_score(results[results['sample'] == 'kpi']['y_true'], results[results['sample'] == 'kpi']['y_pred_without_dist_and_tech'])

0.9316770186335404

In [ ]:
recall_score(results[results['sample'] == 'kpi']['y_true'], results[results['sample'] == 'kpi']['y_pred_without_dist_and_'])